In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import callbacks
from keras.callbacks import ModelCheckpoint
from pathlib import Path
import os.path
from datetime import datetime
from sklearn.cluster import KMeans
from keras import layers
import pandas as pd

In [ ]:
epochs=100
batch_size = 54
img_height = 240
img_width = 320
data_dir="data/img/genre_detector"
dataset = tf.keras.preprocessing.image_dataset_from_directory(data_dir, image_size=(img_height,img_width), batch_size=batch_size,labels="inferred",crop_to_aspect_ratio=True)
class_names = dataset.class_names

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset="training",seed=42,image_size=(img_height, img_width),batch_size=batch_size,crop_to_aspect_ratio=True)
val_ds = tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset="validation",seed=42,image_size=(img_height, img_width),batch_size=batch_size,crop_to_aspect_ratio=True)
class_names = train_ds.class_names
num_classes = len(class_names)

In [ ]:
def faz_model(optimizer="adam",metrics=["accuracy"]):
    model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255),
        #primeira camada convolucional
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        #segunda camada convolucional
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        #terceira camada convolucional
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        #quarta camada convolucional
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        #camada densa
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(num_classes)])
    model.compile(optimizer=optimizer,loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=metrics)
    return model

In [ ]:
n_init=10
checa=0
best_layer=-1
history=[]
current_working_directory = Path.cwd()
model_checkpoint_file = os.path.join(current_working_directory,datetime.now().strftime("%Y%m%d%H%M%S") + '.h5')
checkpoint = ModelCheckpoint(model_checkpoint_file,
                                     monitor='val_accuracy',
                                     verbose=False,
                                     save_best_only=True,
                                     mode='max')
early_stop=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=10,restore_best_weights=True)


for idx in range(n_init):
    print(f"Treino numero: {idx}")
    model=faz_model()
    history.append((model.fit(train_ds,validation_data=val_ds,epochs=epochs,callbacks=[early_stop,checkpoint],verbose=False),model))
    teste=model.evaluate(val_ds)
    if(checa<teste[1]):
        best_layer=idx
        checa=teste[1]

In [ ]:
history_frame = pd.DataFrame(history[best_layer][0].history)
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()
plt.grid(True)

In [ ]:
history[best_layer][1].evaluate(val_ds)